<a href="https://colab.research.google.com/github/thirza258/freecodecamp_rock_paper_scissors/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-01-04 16:46:46--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  59.0MB/s    in 0.4s    

2025-01-04 16:46:47 (59.0 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# Count the number of ratings for each user
user_rating_counts = df_ratings.groupby('user').size()

# Remove users who have rated fewer than 200 books
df_ratings_filtered = df_ratings[df_ratings['user'].isin(user_rating_counts[user_rating_counts >= 200].index)]

# Count the number of ratings for each book
book_rating_counts = df_ratings_filtered.groupby('isbn').size()

# Remove books that have fewer than 100 ratings
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(book_rating_counts[book_rating_counts >= 100].index)]

# Merge the filtered ratings with the books dataframe to get the final dataset
df_combined = pd.merge(df_ratings_filtered, df_books, on='isbn')

# Group by 'title' and aggregate the 'user' and 'rating' columns
# Concatenate users and average ratings for each book
df_grouped = df_combined.groupby('title').agg(
    users=('user', lambda x: ','.join(map(str, x))),
    avg_rating=('rating', 'mean')
).reset_index()


# Apply TFIDFVectorizer to the titles (with custom parameters)
tfidf_vector = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), min_df=5, max_df=0.9)
tfidf_matrix = tfidf_vector.fit_transform(df_grouped['title'])

svd = TruncatedSVD(n_components=7, random_state=42) # Changed n_components to 7
tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix)

# Create the NearestNeighbors model with better parameters
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
model.fit(tfidf_matrix_reduced)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [ ]:
print(df_grouped['title'].duplicated().sum())

0


In [ ]:
print(df_grouped.head())

                                               title  \
0                                1st to Die: A Novel   
1  A Is for Alibi (Kinsey Millhone Mysteries (Pap...   
2                                 A Map of the World   
3                                    A Painted House   
4                            A Prayer for Owen Meany   

                                               users  avg_rating  
0  6543,8936,11676,14521,16795,18401,20115,21014,...    2.472441  
1  2766,6575,11601,11676,14422,16634,23288,23768,...    1.828571  
2  6575,8245,11676,12824,16634,16795,17950,21014,...    1.338983  
3  278418,9856,10447,11676,13552,14521,16795,2020...    1.723926  
4  277427,278418,7346,9856,11676,13082,13273,1442...    2.007299  


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity between all books
cosine_sim = cosine_similarity(tfidf_matrix)

# Function to get recommendations based on cosine similarity
def get_recommends(book=""):
    # Ensure the book exists in the dataset
    if book not in df_books['title'].values:
        return f"Book '{book}' not found in the dataset."

    # Get the index of the book in the dataframe
    query_index = df_grouped[df_grouped['title'] == book].index[0]

    # Get the pairwise cosine similarities for the book
    similarity_scores = list(enumerate(cosine_sim[query_index]))

    # Sort the books based on their similarity to the input book
    sorted_similarities = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    recommended_books = []
    for i in range(1, 6):  # Return top 5 recommendations (excluding the book itself)
        book_idx = sorted_similarities[i][0]
        recommended_books.append([df_grouped['title'][book_idx], sorted_similarities[i][1]])

    return [book, recommended_books]



In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["Vinegar Hill (Oprah's Book Club (Paperback))", 1.0], ["Where the Heart Is (Oprah's Book Club (Paperback))", 1.0], ["The Book of Ruth (Oprah's Book Club (Paperback))", 0.9564208380158772], ["She's Come Undone (Oprah's Book Club)", 0.9166805702337016], ["White Oleander : A Novel (Oprah's Book Club)", 0.857061708930007]]]
You haven't passed yet. Keep trying!
